In [1]:
import sys
sys.path.append('..')

import numpy as np
import scipy
import mir_eval
import torch
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import datasets.feature_extract as fe
import datasets.randomepisode as re
import datasets.activeepisode as ae
import datasets.data_gen as dg
import datasets.smoothquery as sq
import datasets.specaugmentset as sas
import models.prototypical as pt
from sklearn.cluster import  KMeans

import pandas as pd
import os
import utils
import math
import json
import csv
from datetime import datetime
import copy
from scipy import stats
from glob import glob
from collections import defaultdict

import scipy

from multiprocessing import Pool

In [2]:
#

### Post processing entry

In [13]:
def post_processing():
    
    predictions_file = open('postprocessing/G_TEST_out.csv', newline='')
    new_predictions_file = 'postprocessing/G_PP_TEST_out.csv'
    
    predictions_reader = csv.reader(predictions_file, delimiter=',')
    predictions = []
    for e in predictions_reader:
        predictions.append(e)
    
    #Median filtering
    predictions = median_filtering(predictions)
    #Remove short duration events 
    predictions = remove_short(predictions)
    
    
    with open(new_predictions_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(predictions)

post_processing()

Median filtering


100%|██████████████████████████████████████████████████████████████████████████████| 31/31 [03:17<00:00,  6.38s/it]


### Median filter

In [12]:
def median_filtering(predictions):
    
    #Number of ticks per second
    rate = 12000
    increment = 1.0/rate
    
    num_proc=16
    pool = Pool(processes=num_proc)
    
    #dict with prediction onset and offsets split over audiofiles
    pred_dict = defaultdict(list)
    
    #Skip header
    for i in range(1,len(predictions)):
        pred_dict[predictions[i][0]].append([predictions[i][1], predictions[i][2]])
    
    new_predictions = []
    #Attach header
    new_predictions.append(predictions[0])
    
    print('Median filtering')
    
    for key in tqdm(pred_dict.keys()):
        
        #Go from onset offset predictions to array with binary elements indicating events
        index = 0.0
        prediction_array = []
        for event in pred_dict[key]:
            while index < float(event[0]):
                prediction_array += [0]
                index += increment
            while index < float(event[1]):
                prediction_array += [1]
                index += increment
        
        #How should we think around this window size?
        #Can we base it per file?
        data_test = '/home/willbo/data/dcase/test'
        n_shots = 5
        csv_files = [file for file in glob(os.path.join(data_test, '*.csv'))]
        dict_duration = {}
        for csv_file in csv_files:
            audiofile = csv_file.replace('.csv', '.wav')
            df = pd.read_csv(csv_file)
            events = dummy_choice(df, n_shots)
            duration = 0.0 #configurable?
            for event in events:
                duration += float(event[2])-float(event[1])
                    
            #dict_duration[audiofile] = min_duration
            #One third of the average duration of the shots. Use this as window lenght, ezi pizi.
            #In seconds
            dict_duration[os.path.split(audiofile)[1]] = 0.33*(duration/n_shots)
        #print('Using window size: '+str(dict_duration[key])+' for file: '+key)
        w_size = dict_duration[key] #in s, whole size of window, so from left of index to right = 0.05s
        
        prediction_array = np.array(prediction_array)
        w_size = math.floor(w_size/increment)
        
        if w_size % 2 == 0:
            w_size += 1
        
        d = math.floor(len(prediction_array)/num_proc)
        
        '''
        TODO: Obs this method brings us extremely close to mem error. This is not really a sustainable approach.
        We need to do some mem management code here probably.
        It looks like the notebook stalls.
        Chaning the rate to 12000 seems to do the trick. Should still be granular enough.
        '''
        
        ixs = []
        for i in range(num_proc):
            if i == (num_proc-1):
                ixs += [list(range(i*d-int((w_size-1)/2), len(prediction_array)))]
            else:    
                ixs += [list(range(max(0, i*d-int((w_size-1)/2)), (i+1)*d+int((w_size-1)/2)))]

        worker_output = pool.map(worker_filter_array, [(int(w_size), prediction_array[ix]) for ix in ixs])
        w_filtered_array = []
        
        for i in range(len(worker_output)):
            if i == 0:
                w_filtered_array += [worker_output[i][0:len(worker_output[i])-int((w_size-1)/2)]]
            elif i == len(worker_output)-1:
                w_filtered_array += [worker_output[i][int((w_size-1)/2):len(worker_output[i])]]
            else:
                w_filtered_array += [worker_output[i][int((w_size-1)/2):len(worker_output[i])-int((w_size-1)/2)]]
        
        
        #TODO: Take care of warning here. This is deprecated. Gotta flatten it outside of numpy i think!
        filtered_array = np.zeros(0)
        for e in w_filtered_array:
            filtered_array = np.append(filtered_array, e)
        #filtered_array = median_filter_array(prediction_array, math.floor(w_size/(increment)))
        #Reverse from array with binary elements indicating events to onset offset predictions
        krn = [1, -1]
        changes = np.convolve(filtered_array, krn)
        #This is deprecated and fails for some reason?
        onsets = np.where(changes == 1)
        offsets = np.where(changes == -1)
        events = list(zip(onsets[0], offsets[0]))
        new_events = []
        for e in events:
            new_events.append([e[0]*increment, e[1]*increment])
        pred_dict[key] = new_events
        
    
    #Assume new predictions to be in a dictionary with files as keys and elements being [onset, offset]
    for key in pred_dict.keys():
        for e in pred_dict[key]:
            new_predictions.append([key, e[0], e[1]])
    
    return new_predictions

def median_filter_array(array, window_size):
    return scipy.ndimage.median_filter(array, size=window_size, mode='nearest')

def worker_filter_array(t):
    return scipy.ndimage.median_filter(t[1], size=t[0], mode='nearest')

### Remove short duration events

In [3]:
def dummy_choice(csv, n_shots):
    events = []
    for i in range(len(csv)):
                if(csv.loc[i].values[-1] == 'POS' and len(events) < n_shots):
                    events.append(csv.loc[i].values)
    return events

In [4]:
def remove_short(predictions):
    data_test = '/home/willbo/data/dcase/test'
    n_shots = 5
    
    csv_files = [file for file in glob(os.path.join(data_test, '*.csv'))]
    dict_duration = {}
    for csv_file in csv_files:
        audiofile = csv_file.replace('.csv', '.wav')
        df = pd.read_csv(csv_file)
        events = dummy_choice(df, n_shots)
        min_duration = 10000 #configurable?
        for event in events:
            if float(event[2])-float(event[1]) < min_duration:
                min_duration = float(event[2])-float(event[1])
        #dict_duration[audiofile] = min_duration
        dict_duration[os.path.split(audiofile)[1]] = min_duration
    
    '''
    results = []
    with open(evaluation_file, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader, None)  # skip the headers
        for row in reader:
            results.append(row)
    '''
    
    new_results = [['Audiofilename', 'Starttime', 'Endtime']]
    for event in predictions[1:]:
        audiofile = os.path.split(event[0])[1]
        min_dur = dict_duration[audiofile]
        if float(event[2])-float(event[1]) >= 0.6*min_dur:
            new_results.append([os.path.split(event[0])[1], event[1], event[2]])

    return new_results

In [11]:
kr = [1, -1]
a = [0,0,0,1,1,1,1,0,0,1,1,0]
changes = np.convolve(a, kr)
on = np.where(changes == 1)
off = np.where(changes == -1)
events = list(zip(on[0], off[0]))


t = np.zeros(0)
print(t)
t = np.append(t, np.array([1,1,1,1]))
t = np.append(t, np.array([2,2,2,2]))
print(t)

[]
[1. 1. 1. 1. 2. 2. 2. 2.]
